## AI Integration Services - Customer Outreach Campaign Automation

### This system identifies and reaches out to software companies that could benefit from AI integration services


## Tools

Three key elements of Tools that makes a tool better :
- **Versatility**

- **Fault Tolerance**

- **Caching**

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [6]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
SERPER_API_KEY = userdata.get('SERPER_API_KEY')


In [7]:
from crewai import Agent, Task, Crew

In [9]:
from langchain.chat_models import ChatOpenAI

# Creating Agents

In [11]:
tech_researcher = Agent(
    role="Technical Research Analyst",
    goal="Identify software companies that could benefit from AI integration",
    backstory=(
        "You are an expert in identifying companies that could enhance their "
        "products through AI integration. You analyze their current tech stack, "
        "market position, and growth potential to determine if they're good candidates "
        "for AI services."
      ),
    allow_delegation=False,
    verbose=True,
    llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name='gpt-4o')

)


In [12]:
ai_solutions_consultant = Agent(
    role="AI Solutions Consultant",
    goal="Create compelling proposals for AI integration",
    backstory=(
        "You specialize in crafting tailored AI integration proposals. "
        "With deep knowledge of both AI capabilities and business needs, "
        "you create compelling messages that demonstrate clear value proposition "
        "and ROI potential of AI integration."
    ),
    allow_delegation=False,
    verbose=True,
    llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name='gpt-4o')
)

# Creating Tools


## crewAI tools

In [15]:
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool

In [16]:
directory_read_tool = DirectoryReadTool(directory='/content/company_data')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

## Custom tools

In [17]:
from crewai_tools import BaseTool

In [18]:
# Custom sentiment analysis tool
class SentimentAnalysisTool(BaseTool):
    name: str = "Sentiment Analysis Tool"
    description: str = ("Analyzes message sentiment to ensure professional and engaging communication tone")

    def _run(self, text: str) -> str:
        # In a real implementation, you would add sentiment analysis logic here
        return "positive"  # for simplicity purpose

In [19]:
sentiment_analysis_tool = SentimentAnalysisTool()

# Creating Tasks

In [20]:
company_analysis_task = Task(
    description=(
        "Research {company_name}, a {company_type} company. "
        "Analyze their current technology stack, market position, "
        "and potential areas where AI integration could provide value. "
        "Focus on their {main_product} product and recent "
        "technological developments. Identify specific pain points "
        "that AI could address."
    ),
    expected_output=(
        "A detailed analysis report including:\n"
        "1. Current technology stack\n"
        "2. Market position and competitors\n"
        "3. Specific areas where AI could add value\n"
        "4. Potential ROI from AI integration"
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=tech_researcher,
)

In [21]:
proposal_creation_task = Task(
    description=(
        "Based on the analysis of {company_name}, create a "
        "personalized proposal for {decision_maker}, "
        "their {position}. Focus on how AI integration can "
        "enhance their {main_product} and address their "
        "specific challenges. Reference their recent "
        "{recent_achievement} and connect it to potential "
        "AI-driven improvements."
    ),
    expected_output=(
        "A personalized proposal including:\n"
        "1. Executive summary highlighting key benefits\n"
        "2. Specific AI integration suggestions\n"
        "3. Expected ROI and implementation timeline\n"
        "4. Next steps and call to action"
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=ai_solutions_consultant,
)

# Creating The Crew

In [25]:
crew = Crew(
    agents=[tech_researcher, ai_solutions_consultant],
    tasks=[company_analysis_task, proposal_creation_task],
    verbose=2,
    # memory=True
)

# Running the Crew

In [26]:
inputs = {
        "company_name": "TechStack Solutions",
        "company_type": "Developer Tools",
        "main_product": "CI/CD Platform",
        "decision_maker": "Sarah Chen",
        "position": "CTO",
        "recent_achievement": "launch of containerized deployment feature"
    }

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Technical Research Analyst
 [INFO]: == Starting Task: Research TechStack Solutions, a Developer Tools company. Analyze their current technology stack, market position, and potential areas where AI integration could provide value. Focus on their CI/CD Platform product and recent technological developments. Identify specific pain points that AI could address.


> Entering new CrewAgentExecutor chain...
To begin the analysis of TechStack Solutions, I need to gather information about the company, particularly their technology stack, market position, and recent technological developments. I will start by listing the files in the directory that might contain relevant information.

Action: List files in directory
Action Input: {} 

File paths: 
-/content/company_data/techstack_solutions.md
- /content/company_data/cloudflow_analytics.md

Thought: I have found a file named "techstack_solutions.md" which likely contains information about TechStack Solutions. I will re

In [27]:
from IPython.display import Markdown
Markdown(result)

Dear Sarah Chen,

Executive Summary:
TechStack Solutions has made significant strides in the CI/CD space, particularly with the recent launch of your containerized deployment feature. This proposal outlines how integrating AI into your CI/CD platform can further enhance your capabilities, addressing specific challenges such as test management complexity, predictive analytics for deployment, resource optimization, and automated security scanning. By leveraging AI, TechStack Solutions can achieve efficiency gains, cost reductions, enhanced security, and a competitive edge in the developer tools sector.

Specific AI Integration Suggestions:
1. AI-Driven Test Management: Implement AI systems to optimize testing processes by identifying critical test cases, reducing redundancies, and predicting outcomes, leading to faster and more reliable deployments.
2. Predictive Analytics for Deployment: Utilize AI to analyze historical data and performance metrics, predicting and mitigating potential deployment issues proactively.
3. Resource Optimization: Deploy AI algorithms for dynamic resource allocation, ensuring optimal utilization and cost savings.
4. Automated Security Enhancements: Integrate AI for real-time threat detection and automated responses, strengthening your security posture.

Expected ROI and Implementation Timeline:
- Efficiency Gains: Anticipated reduction in time-to-market by 20% within the first year.
- Cost Reduction: Projected operational cost savings of 15% annually.
- Enhanced Security: Decrease in security incidents by 30%, minimizing potential financial and reputational risks.
- Implementation Timeline: A phased approach over 12 months, starting with test management and predictive analytics, followed by resource optimization and security enhancements.

Next Steps and Call to Action:
I recommend scheduling a meeting to discuss this proposal in detail and outline a tailored implementation plan. Please let me know your availability for a meeting next week.

Looking forward to the opportunity to support TechStack Solutions in achieving even greater success.

Best regards,

[Your Name]  
AI Solutions Consultant

## BaseTool Class

The **BaseTool** class in CrewAI is a foundational component that provides a standardized framework for building any tool you want your AI agents to use. Essentially, it’s an abstract base class that defines the interface and core behavior for all tools within the framework. By extending this class, developers can easily create custom tools with consistent behavior, robust input validation, and built-in error handling.

---

## 1. What Is the BaseTool Class?

### Role in CrewAI
- **Standardization:** The BaseTool class defines a standard interface (e.g., methods and attributes) that all tools must follow. This ensures that regardless of what the tool does, it can be called in the same way by any agent.

- **Error Handling:** It often includes built-in error handling so that if something goes wrong during the execution of a tool, the framework can handle the error gracefully.

- **Input Validation:** By using an input schema (typically defined with Pydantic), BaseTool helps validate the inputs provided to the tool. This minimizes the risk of errors due to unexpected input types or missing data.

- **Integration:** Since all custom tools inherit from BaseTool, they automatically integrate with the CrewAI ecosystem—meaning agents can seamlessly use these tools as part of their task workflows.

### Key Components of BaseTool
Although you might not see every detail (as some parts may be abstracted away in the framework), the BaseTool class typically includes:

- **Attributes:**
  - `name`: A string that identifies the tool.

  - `description`: A brief description of what the tool does.

  - `args_schema`: A Pydantic model that defines the expected inputs for the tool.

- **Methods:**
  - `run()`: A public method that wraps the internal logic. It usually handles input validation (using `args_schema`) and calls the tool’s custom logic.

  - `_run()`: An abstract or placeholder method that you must override in your subclass to implement the tool’s specific functionality.

---

## 2. How BaseTool Helps in Creating Custom Tools

### Simplification
When you subclass BaseTool, you only need to implement the `_run()` method that contains your tool’s logic. The BaseTool class takes care of:

- Validating the input using the schema you provide.

- Managing exceptions so that errors are caught and handled in a uniform way.

- Integrating your tool into the CrewAI workflow, making it available to agents without further configuration.

### Consistency
By having all tools derive from BaseTool, every tool in your project will have a similar structure and behavior. This consistency simplifies:

- **Maintenance:** Debugging or upgrading tools is easier when they follow the same pattern.

- **Interoperability:** Agents don’t need to worry about how a tool is implemented—they just call its `run()` method and expect a consistent type of output.

---

## 3. Detailed Code Examples

### Example 1: Creating a Custom Tool by Subclassing BaseTool

Below is an example that demonstrates how to create a custom search tool by subclassing BaseTool.

```python
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type

# Define an input schema for the tool using Pydantic
class SearchToolInput(BaseModel):
    query: str = Field(..., description="The search query for which results are needed.")

# Create a custom tool by subclassing BaseTool
class CustomSearchTool(BaseTool):
    name: str = "CustomSearchTool"
    description: str = "A tool that performs a custom search and returns simulated results."
    args_schema: Type[BaseModel] = SearchToolInput  # Input validation schema

    # Implement the _run method with your custom logic.
    def _run(self, query: str) -> str:
        # Here, you would normally implement the actual search logic.
        # For demonstration, we return a dummy string.
        return f"Simulated search results for '{query}'"

# Instantiate and use the custom tool
custom_tool = CustomSearchTool()
result = custom_tool.run(query="CrewAI custom tools")
print(result)
# Output: Simulated search results for 'CrewAI custom tools'
```

**Explanation:**
1. **SearchToolInput:** This Pydantic model ensures that any input to the tool contains a valid `query` string.
2. **CustomSearchTool:**  
   - It defines the `name` and `description` attributes that help identify the tool in the CrewAI ecosystem.
   - The `args_schema` is set to `SearchToolInput` so that inputs are automatically validated.
   - The `_run()` method contains the core functionality; in this example, it simulates a search operation.
3. **Usage:** Calling the `run()` method on the tool instance first validates the input against the schema and then calls `_run()` to produce the output.

### Example 2: Creating a Simple Tool Using the @tool Decorator

For simpler scenarios where a full class definition is unnecessary, you can convert a function into a tool with a decorator.

```python
from crewai.tools import tool

@tool("QuickSearch")
def quick_search(query: str) -> str:
    """
    A simple tool that simulates a search operation.
    Returns a dummy result for the provided query.
    """
    # In an actual implementation, you might integrate API calls or other logic.
    return f"Quick results for '{query}'"

# Use the decorated function as a tool
print(quick_search("CrewAI BaseTool explanation"))
# Expected output: Quick results for 'CrewAI BaseTool explanation'
```

**Explanation:**
- The `@tool` decorator wraps the `quick_search` function, converting it into a tool with the name "QuickSearch."

- This method is concise and is suitable when you need to implement a straightforward operation.

---

## 4. Benefits Recap

- **Rapid Development:** By inheriting from BaseTool, you reduce the amount of boilerplate code you need to write.  

- **Input Safety:** Automatic input validation via the `args_schema` ensures your tool receives correctly structured data.

- **Error Management:** Built-in error handling in the BaseTool class helps manage exceptions, making your tools more robust.

- **Seamless Integration:** All custom tools follow the same interface, so they integrate easily with CrewAI’s agent and task systems.

---

For further reading, you can refer to the [CrewAI Tools Documentation](https://docs.crewai.com/core-concepts/tools).